#### Import pymongo

In [1]:
from pymongo import MongoClient
from  pprint import pprint

#### Connect to MongoDB

In [2]:
client = MongoClient(host='localhost',port=27017)

#### Create Database 

In [3]:
dbcatering = client['catering']

#### Create Collections

In [4]:
Customers = dbcatering['customers']
Foods = dbcatering['foods']
Orders = dbcatering['orders']

#### insert document in customers  collection

In [63]:
Customers.insert_many([
    {'_id':1,'Name':'Ali','Family':'Bahrami','Active':True,'MobileNumber':'09370589962','Address':'Malayer kh Emam'},
    {'_id':2,'Name':'Reza','Family':'Akbari','Active':False,'MobileNumber':'09388925364'},
    {'_id':3,'Name':'Ahmad','Family':'Rezaei','Active':True,'MobileNumber':'09373653214','Address':'Malayer kh Arak'},
    {'_id':4,'Name':'Javad','Family':'Mohamadi','Active':True,'MobileNumber':'09125642145'},
    {'_id':5,'Name':'Sara','Family':'Akbari','Active':True,'MobileNumber':'09182086524'},
    {'_id':6,'Name':'Negin','Family':'Moeini','Active':True,'MobileNumber':'09386405891','Address':'Malayer kh khayam'},
    {'_id':7,'Name':'Sajad','Family':'Darvishi','Active':False,'MobileNumber':'09915831540'},
    {'_id':8,'Name':'Ali','Family':'Tehrani','Active':True,'MobileNumber':'09213605941'},
    {'_id':9,'Name':'Mehdi','Family':'Abbasi','Active':True,'MobileNumber':'09301597565'},
    {'_id':10,'Name':'Ayda','Family':'Javadi','Active':True,'MobileNumber':'09189513570','Address':'Malayer kh Shohada'},
])

#### insert document in foods collection

In [62]:
Foods.insert_many([
    {'_id':1001,'name':'Pizza','Price':250},
    {'_id':1002,'name':'Meat Kebab','Price':300},
    {'_id':1003,'name':'Stew','Price':100},
    {'_id':1004,'name':'Hotdog','Price':180},
    {'_id':1005,'name':'Pasta','Price':90},
    {'_id':1006,'name':'Sandwich','Price':120},
    {'_id':1007,'name':'Bacon','Price':210},
    {'_id':1008,'name':'Hamburger','Price':150},
    {'_id':1009,'name':'Cheeseburger','Price':190},
    {'_id':1010,'name':'Fried Chicken','Price':280},
])

#### insert document in foods collection

In [13]:
Orders.insert_many([
    {'_id':101,'CustomerCode':8,'FoodCode':[1001,1003,1007],'Date':'2023-07-01','send':True,'with food':['Soft drinks','yogurt']},
    {'_id':102,'CustomerCode':5,'FoodCode':1006,'Date':'2023-07-01','with food':['sauce']},
    {'_id':103,'CustomerCode':6,'FoodCode':1002,'Date':'2023-07-02','with food':['Salad','yogurt']},
    {'_id':104,'CustomerCode':1,'FoodCode':1003,'Date':'2023-07-03','send':True},
    {'_id':105,'CustomerCode':3,'FoodCode':1006,'Date':'2023-07-03','with food':['Soft drinks']},
    {'_id':106,'CustomerCode':1,'FoodCode':1009,'Date':'2023-07-05',},
    {'_id':107,'CustomerCode':1,'FoodCode':1005,'Date':'2023-07-05','send':True},
    {'_id':108,'CustomerCode':3,'FoodCode':[1004,1003],'Date':'2023-07-05','with food':['Soft drinks','sauce']},
    {'_id':109,'CustomerCode':6,'FoodCode':[1007,1002,1010],'Date':'2023-07-06','with food':['Olive']},
    {'_id':110,'CustomerCode':9,'FoodCode':1006,'Date':'2023-07-07','send':True},
    {'_id':111,'CustomerCode':4,'FoodCode':1010,'Date':'2023-07-07'},
    {'_id':112,'CustomerCode':5,'FoodCode':1010,'Date':'2023-07-07'},
    {'_id':113,'CustomerCode':9,'FoodCode':1009,'Date':'2023-07-08','send':True},
    {'_id':114,'CustomerCode':4,'FoodCode':1001,'Date':'2023-07-08','with food':['Soft drinks','sauce']},
    {'_id':115,'CustomerCode':8,'FoodCode':1001,'Date':'2023-07-08'},
    {'_id':116,'CustomerCode':4,'FoodCode':[1010,1002],'Date':'2023-07-08'}
])

### Query1

In [14]:
for doc in Foods.find({},{'_id':1,'name':1,'Price':1}).sort('Price',-1):
    print(doc)

### Query2

In [15]:
for doc in Customers.find({'Active':True},{'_id':0,'Address':0}):
    print(doc)

### Query3

In [16]:
for doc in Foods.find({'$and':[{'Price':{'$gte':120}},{'Price':{'$lte':260}}]},{'_id':0,'name':1,'Price':1}).sort('Price'):
    print(doc)

### Query4

In [17]:
for doc in Orders.find({'Date':'2023-07-08'},{'send':0,'Price':0}):
    print(doc)

### Query5

In [22]:
for item in Orders.aggregate([
    {'$match':{'Date':'2023-07-07'}},
    {'$unwind':'$FoodCode'},
    {'$lookup':{'from':'foods','localField':'FoodCode','foreignField':'_id','as':'namefood'}},
    {'$unwind':'$namefood'},
    {'$group':{'_id':'$FoodCode','count':{'$count':{}},'totalPrice':{'$sum':'$namefood.Price'}}}
    ]): 
        pprint(item)


### Query6

In [23]:
for item in Orders.aggregate([{'$match':{'CustomerCode':4}}, 
    {'$lookup':{'from':'foods','localField':'FoodCode','foreignField':'_id','as':'namefood'}},
    {'$unwind':'$namefood'},
    {'$group':{'_id':'$CustomerCode','totalOrders':{'$sum':1},'foodsOrdered':{'$push':'$namefood.name'}}},
    ]): 
        pprint(item)